# **Emoticon Dataset**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, confusion_matrix
from tensorflow.keras.models import Sequential

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import requests
np.random.seed(0)

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dense, Dropout, Input, Embedding
from keras.optimizers import Adam

# Read emoticon dataset
train_emoticon_df = pd.read_csv("https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/train/train_emoticon.csv")
train_emoticon_X = train_emoticon_df['input_emoticon'].tolist()
train_emoticon_Y = train_emoticon_df['label'].tolist()

valid_emoticon_df = pd.read_csv("https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/valid/valid_emoticon.csv")
valid_emoticon_X = valid_emoticon_df['input_emoticon'].tolist()
valid_emoticon_Y = valid_emoticon_df['label'].tolist()

test_emoticon_X = pd.read_csv("https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/test/test_emoticon.csv")['input_emoticon'].tolist()

# Convert lists to NumPy arrays
train_emoticon_X = np.array(train_emoticon_X)
valid_emoticon_X = np.array(valid_emoticon_X)
test_emoticon_X = np.array(test_emoticon_X)

# Ensure labels are NumPy arrays
train_emoticon_Y = np.array(train_emoticon_Y)
valid_emoticon_Y = np.array(valid_emoticon_Y)

# Concatenate all texts to get unique characters
all_texts = ''.join(train_emoticon_X.flatten()) + ''.join(valid_emoticon_X.flatten()) + ''.join(test_emoticon_X.flatten())
unique_characters = np.unique(list(all_texts))

# Create character-to-index mapping
char_to_index = {char: idx + 1 for idx, char in enumerate(unique_characters)}  # Index starts from 1 for embedding
num_classes = len(unique_characters) + 1  # Include 0 for padding

# Convert sequences to integer sequences
def convert_to_sequences(sequences):
    return [[char_to_index[char] for char in seq] for seq in sequences]

# Convert to integer sequences
train_emoticon_X = convert_to_sequences(train_emoticon_X)
valid_emoticon_X = convert_to_sequences(valid_emoticon_X)
test_emoticon_X = convert_to_sequences(test_emoticon_X)

# Pad sequences to a fixed length
max_len = 13  # Fixed sequence length
train_emoticon_X = pad_sequences(train_emoticon_X, maxlen=max_len, padding='post')
valid_emoticon_X = pad_sequences(valid_emoticon_X, maxlen=max_len, padding='post')
test_emoticon_X = pad_sequences(test_emoticon_X, maxlen=max_len, padding='post')

In [2]:
# Define the model for feature extraction
embedding_dimension = 12 # Dimension of the embedding
# Define and compile the model
model = Sequential()
model.add(Embedding(input_dim=num_classes, output_dim=embedding_dimension))  # Added input_length
model.add(Flatten())
model.add(Dense(24, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(train_emoticon_X, train_emoticon_Y, epochs=10, batch_size=32, validation_data=(valid_emoticon_X, valid_emoticon_Y))


# Predictions and calculate F1 score and confusion matrix
valid_predictions_emoticon = (model.predict(valid_emoticon_X) > 0.5).astype("int32")
train_predictions_emoticon = (model.predict(train_emoticon_X) > 0.5).astype("int32")
test_predictions_emoticon = (model.predict(test_emoticon_X) > 0.5).astype("int32")

valid_predictions_emoticon = valid_predictions_emoticon.flatten()
train_predictions_emoticon = train_predictions_emoticon.flatten()
test_predictions_emoticon = test_predictions_emoticon.flatten()

f1 = f1_score(valid_emoticon_Y, valid_predictions_emoticon)

# Calculate validation accuracy
valid_accuracy_emoticon = accuracy_score(valid_emoticon_Y, valid_predictions_emoticon)
# Calculate training accuracy
train_accuracy_emoticon = accuracy_score(train_emoticon_Y, train_predictions_emoticon)

model.summary()

print(f"Training Accuracy for emoticon: {train_accuracy_emoticon:.4f}")
print(f"Validation Accuracy for emoticon: {valid_accuracy_emoticon:.4f}")
print(f"F1 Score for emoticon: {f1:.4f}")

Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5227 - loss: 0.6902 - val_accuracy: 0.8937 - val_loss: 0.5502
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9270 - loss: 0.3790 - val_accuracy: 0.9632 - val_loss: 0.1451
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9724 - loss: 0.1206 - val_accuracy: 0.9714 - val_loss: 0.0974
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9824 - loss: 0.0757 - val_accuracy: 0.9734 - val_loss: 0.0756
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9858 - loss: 0.0637 - val_accuracy: 0.9796 - val_loss: 0.0652
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9906 - loss: 0.0497 - val_accuracy: 0.9816 - val_loss: 0.0552
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9916 - loss: 0.0410 - val_accuracy: 0.9816 - val_loss: 0.0501
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9915 - loss: 0.0372 - val_accuracy: 0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 13, 12)              │           2,724 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 156)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 24)                  │           3,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              25 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,553 (76.38 KB)

 Trainable params: 6,517 (25.46 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 13,036 (50.93 KB)

Training Accuracy for emoticon: 0.9982
Validation Accuracy for emoticon: 0.9796
F1 Score for emoticon: 0.9785


In [3]:
print(test_predictions_emoticon)

[1 1 0 ... 1 1 0]


# **Features Dataset**

In [4]:
# URL of the raw .npz file
url_train = "https://github.com/psarath22/CS771_M1/raw/main/datasets/train/train_feature.npz"
url_valid = "https://github.com/psarath22/CS771_M1/raw/main/datasets/valid/valid_feature.npz"
url_test = "https://github.com/psarath22/CS771_M1/raw/main/datasets/test/test_feature.npz"
# Download the file
response1 = requests.get(url_train)
response2= requests.get(url_valid)
response3 = requests.get(url_test)

# Save the content to a file
with open('train_feature.npz', 'wb') as f:
    f.write(response1.content)
    print("Downloading TrainData completed!")

# Save the content to a file
with open('valid_feature.npz', 'wb') as f:
    f.write(response2.content)
    print("Downloading validData completed!")

# Save the content to a file
with open('test_feature.npz', 'wb') as f:
    f.write(response3.content)
    print("Downloading TestData completed!")


# read feat dataset
train_feat = np.load("train_feature.npz", allow_pickle=True)
train_feat_X = train_feat['features']
train_feat_Y = train_feat['label']

test_feat_X = np.load("test_feature.npz", allow_pickle=True)['features']

valid_feat = np.load("valid_feature.npz", allow_pickle=True)
valid_feat_X = valid_feat['features']
valid_feat_Y = valid_feat['label']


print(f"Train dataset size: ")
print(f"train_feat_X: {len(train_feat_X)} train_feat_Y: {len(train_feat_Y)}")

print()
print("Valid dataset size: ")
print(f"valid_feat_X: {len(valid_feat_X)} valid_feat_Y: {len(valid_feat_Y)}")

print()
print("Test dataset size: ")
print(f"test_feat_X: {len(test_feat_X)}")


Train dataset size: 
train_feat_X: 7080 train_feat_Y: 7080

Valid dataset size: 
valid_feat_X: 489 valid_feat_Y: 489

Test dataset size: 
test_feat_X: 2232


In [5]:
# Create the training and validation  dataset

train_feat_X = np.array([feat for feat in train_feat_X])
print(train_feat_X.shape)
valid_feat_X = np.array([feat for feat in valid_feat_X])
print(valid_feat_X.shape)
test_feat_X = np.array([feat for feat in test_feat_X])
print(test_feat_X.shape)

train_feat_X = np.array([feat.flatten() for feat in train_feat_X])
valid_feat_X = np.array([feat.flatten() for feat in valid_feat_X])
test_feat_X = np.array([feat.flatten() for feat in test_feat_X])

n_components = 150
pca = PCA(n_components=n_components)
train_feat_X = pca.fit_transform(train_feat_X)
valid_feat_X = pca.transform(valid_feat_X)
test_feat_X = pca.transform(test_feat_X)
print(train_feat_X.shape)
print(valid_feat_X.shape)
print(test_feat_X.shape)

(7080, 13, 768)
(489, 13, 768)
(2232, 13, 768)
(7080, 150)
(489, 150)
(2232, 150)


In [6]:
# Train Logistic Regression classifier
logreg_classifier = LogisticRegression(max_iter=1000)
logreg_classifier.fit(train_feat_X, train_feat_Y)

# Predict classes for validation data
valid_predictions_feat = logreg_classifier.predict(valid_feat_X)
train_predictions_feat = logreg_classifier.predict(train_feat_X)
test_predictions_feat = logreg_classifier.predict(test_feat_X)

valid_predictions_feat = valid_predictions_feat.flatten()
train_predictions_feat = train_predictions_feat.flatten()
test_predictions_feat = test_predictions_feat.flatten()

# Calculate validation accuracy
valid_accuracy_feat = accuracy_score(valid_feat_Y, valid_predictions_feat)
print(f"Validation Accuracy with Logistic Regression for features dataset: {valid_accuracy_feat}")

# Calculate training accuracy
train_accuracy_feat = accuracy_score(train_feat_Y, train_predictions_feat)
print(f"Train Accuracy with Logistic Regression for features dataset: {train_accuracy_feat}")

f1 = f1_score(valid_feat_Y, valid_predictions_feat, average='weighted')
print(f"f1 score with logistic Regression  for features dataset: {f1}")

# Calculate the number of trainable parameters
n_weights = logreg_classifier.coef_.size  # Number of weights (features)
n_intercepts = logreg_classifier.intercept_.size  # Number of intercepts (bias term)

n_trainable_parameters = n_weights + n_intercepts
print(f"Number of trainable parameters  for features dataset: {n_trainable_parameters}")


Validation Accuracy with Logistic Regression for features dataset: 0.9877300613496932
Train Accuracy with Logistic Regression for features dataset: 0.9853107344632769
f1 score with logistic Regression  for features dataset: 0.9877300613496932
Number of trainable parameters  for features dataset: 151


In [7]:
print(test_predictions_feat)

[1 1 0 ... 0 1 1]


# **Text Sequence Dataset**

In [8]:
# read text sequence dataset
train_seq_df = pd.read_csv('https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/train/train_text_seq.csv')
train_seq_X = train_seq_df['input_str'].tolist()
train_seq_Y = train_seq_df['label'].tolist()

test_seq_X = pd.read_csv("https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/test/test_text_seq.csv")['input_str'].tolist()

# read text sequence dataset
valid_seq_df = pd.read_csv("https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/valid/valid_text_seq.csv")
valid_seq_X = valid_seq_df['input_str'].tolist()
valid_seq_Y = valid_seq_df['label'].tolist()

print(f"Train dataset size: ")
print(f"train_seq_X: {len(train_seq_X)} train_seq_Y: {len(train_seq_Y)}")

print()
print("Valid dataset size: ")
print(f"valid_seq_X: {len(valid_seq_X)} valid_seq_Y: {len(valid_seq_Y)}")

print()
print("Test dataset size: ")
print(f"test_seq_X: {len(test_seq_X)}")

Train dataset size: 
train_seq_X: 7080 train_seq_Y: 7080

Valid dataset size: 
valid_seq_X: 489 valid_seq_Y: 489

Test dataset size: 
test_seq_X: 2232


In [9]:
# Convert list to NumPy array
train_seq_X = np.array(train_seq_X)
valid_seq_X = np.array(valid_seq_X)
test_seq_X = np.array(test_seq_X)
# Convert labels to numpy arrays
train_seq_Y = np.array(train_seq_Y)
valid_seq_Y = np.array(valid_seq_Y)

print(type(train_seq_X))
print(type(train_seq_X[0]))
# Convert list of strings (2D array) to a single string by flattening the array and joining all elements
all_characters = ''.join(train_seq_X.flatten())

# Get the unique characters
unique_characters = np.unique(list(all_characters))

print(f"Unique characters: {unique_characters}")
print(f"Number of unique characters: {len(unique_characters)}")
print(np.unique(valid_seq_Y))

<class 'numpy.ndarray'>
<class 'numpy.str_'>
Unique characters: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']
Number of unique characters: 10
[0 1]


In [10]:
# Get unique characters for two digits
unique_characters = [f"{i:02d}" for i in range(100)]  # ['00', '01', ..., '99']
num_classes = len(unique_characters)  # 100 classes

# Create a mapping from characters to indices
char_to_index = {char: idx for idx, char in enumerate(unique_characters)}

# Function to convert input seqs to a 49x100 matrix using overlapping pairs
def one_hot_count_matrix(seq, max_len=50):
    # Initialize a matrix of zeros
    matrix = np.zeros((max_len - 1, num_classes))  # 49x100 matrix for overlapping pairs
    # Extract overlapping pairs and update the matrix
    for i in range(0, max_len - 1):  # Loop from seq[0:2] to seq[48:50]
        char_pair = seq[i:i + 2]  # Get overlapping pairs of characters
        if char_pair in char_to_index:
            index = char_to_index[char_pair]
            matrix[i, index] = 1  # One-hot encoding
    return matrix.flatten()  # Flatten to a vector for SVM input

# Create the training and validation datasets
train_seq_X = np.array([one_hot_count_matrix(seq) for seq in train_seq_X])
valid_seq_X = np.array([one_hot_count_matrix(seq) for seq in valid_seq_X])
test_seq_X = np.array([one_hot_count_matrix(seq) for seq in test_seq_X])

print(train_seq_X.shape)  # Expected shape: (N_train, 4900)
print(valid_seq_X.shape)  # Expected shape: (N_valid, 4900)
print(test_seq_X.shape) #Expected shape: (N_test, 4900)



(7080, 4900)
(489, 4900)
(2232, 4900)


In [11]:
# Reshape input to fit CNN (samples, timesteps, features)
train_seq_X = train_seq_X.reshape((train_seq_X.shape[0], 49, 100))
valid_seq_X = valid_seq_X.reshape((valid_seq_X.shape[0], 49, 100))
test_seq_X = test_seq_X.reshape((test_seq_X.shape[0], 49, 100))

print(f"train_seq_X shape: {train_seq_X.shape}")
print(f"valid_seq_X shape: {valid_seq_X.shape}")
print(f"test_seq_X shape: {test_seq_X.shape}")


train_seq_X shape: (7080, 49, 100)
valid_seq_X shape: (489, 49, 100)
test_seq_X shape: (2232, 49, 100)


In [12]:
# Define a CNN model with fewer parameters
model = Sequential()
# Input layer
model.add(Input(shape=(49, 100)))
model.add(Conv1D(filters=16, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(16, activation='relu'))  # Smaller dense layer
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(train_seq_X, train_seq_Y, epochs=30, batch_size=32, validation_data=(valid_seq_X, valid_seq_Y))

# Predictions and calculate F1 score and confusion matrix
valid_predictions_seq = (model.predict(valid_seq_X) > 0.5).astype("int32")
train_predictions_seq = (model.predict(train_seq_X) > 0.5).astype("int32")
test_predictions_seq = (model.predict(test_seq_X) > 0.5).astype("int32")
f1 = f1_score(valid_seq_Y, valid_predictions_seq)

valid_predictions_seq = valid_predictions_seq.flatten()
train_predictions_seq = train_predictions_seq.flatten()
test_predictions_seq = test_predictions_seq.flatten()

# Calculate validation accuracy
valid_accuracy_seq = accuracy_score(valid_seq_Y, valid_predictions_seq)
# Calculate training accuracy
train_accuracy_seq = accuracy_score(train_seq_Y, train_predictions_seq)

# Print accuracies and F1 score
model.summary()

print(f"Training Accuracy for sequence data: {train_accuracy_seq:.4f}")
print(f"Validation Accuracy for sequence data: {valid_accuracy_seq:.4f}")
print(f"F1 Score for sequence data: {f1:.4f}")

Epoch 1/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5385 - loss: 0.6775 - val_accuracy: 0.7485 - val_loss: 0.5157
Epoch 2/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7753 - loss: 0.4786 - val_accuracy: 0.8425 - val_loss: 0.3636
Epoch 3/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8358 - loss: 0.3890 - val_accuracy: 0.8569 - val_loss: 0.3081
Epoch 4/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8461 - loss: 0.3532 - val_accuracy: 0.8569 - val_loss: 0.3055
Epoch 5/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8638 - loss: 0.3296 - val_accuracy: 0.8650 - val_loss: 0.2885
Epoch 6/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8637 - loss: 0.3163 - val_accuracy: 0.8650 - val_loss: 0.2730
Epoch 7/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8732 - loss: 0.2912 - val_accuracy: 0.8569 - val_loss: 0.2779
Epoch 8/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8674 - loss: 0.3094 - val_accuracy: 0.

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 48, 16)              │           3,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 24, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 24, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 22, 16)              │             784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 11, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 11, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 176)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │           2,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,549 (80.27 KB)

 Trainable params: 6,849 (26.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 13,700 (53.52 KB)

Training Accuracy for sequence data: 0.9415
Validation Accuracy for sequence data: 0.8814
F1 Score for sequence data: 0.8802


In [13]:
print(test_predictions_seq)

[1 1 0 ... 0 1 0]


# **Combined Data**

In [14]:

# read emoticon dataset
train_emoticon_df = pd.read_csv("https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/train/train_emoticon.csv")
train_emoticon_X = train_emoticon_df['input_emoticon'].tolist()
train_emoticon_Y = train_emoticon_df['label'].tolist()

valid_emoticon_df = pd.read_csv("https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/valid/valid_emoticon.csv")
valid_emoticon_X = valid_emoticon_df['input_emoticon'].tolist()
valid_emoticon_Y = valid_emoticon_df['label'].tolist()

test_emoticon_X = pd.read_csv("https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/test/test_emoticon.csv")['input_emoticon'].tolist()

# Convert list to NumPy array
train_emoticon_X = np.array(train_emoticon_X)
valid_emoticon_X = np.array(valid_emoticon_X)
test_emoticon_X = np.array(test_emoticon_X)

train_emoticon_Y = np.array(train_emoticon_Y)
valid_emoticon_Y = np.array(valid_emoticon_Y)

print(type(train_emoticon_X))
print(type(train_emoticon_X[0]))

# Concatenate all lists into one
all_texts = ''.join(train_emoticon_X.flatten()) + ''.join(valid_emoticon_X.flatten()) + ''.join(test_emoticon_X.flatten())

# Get unique characters
unique_characters = np.unique(list(all_texts))
print(unique_characters)
# Create a mapping from characters to indices
char_to_index = {char: idx for idx, char in enumerate(unique_characters)}
print(unique_characters.shape)
print(len(char_to_index))

# Function to convert input emoticons to a 13x226 matrix
def one_hot_count_matrix(emoticon, max_len=13, num_classes=226):
    # Initialize a matrix of zeros
    matrix = np.zeros((max_len, num_classes))
    # Iterate through the characters in the emoticon
    for i, char in enumerate(emoticon):
        if char in char_to_index:  # Check if char exists in the mapping
            matrix[i, char_to_index[char]] = 1  # One-hot encoding
        else:
            print(f"Character '{char}' not found in char_to_index.")  # Optional: log missing characters
    return matrix.flatten()  # Flatten to a vector for SVM input

# Create the training and validation datasets
train_emoticon_X = np.array([one_hot_count_matrix(emoticon) for emoticon in train_emoticon_X])
valid_emoticon_X = np.array([one_hot_count_matrix(emoticon) for emoticon in valid_emoticon_X])
test_emoticon_X = np.array([one_hot_count_matrix(emoticon) for emoticon in test_emoticon_X])

print(train_emoticon_X.shape)
print(valid_emoticon_X.shape)
print(test_emoticon_X.shape)

<class 'numpy.ndarray'>
<class 'numpy.str_'>
['😀' '😁' '😂' '😃' '😄' '😅' '😆' '😇' '😈' '😉' '😊' '😋' '😌' '😍' '😎' '😏' '😐' '😑'
 '😒' '😓' '😔' '😕' '😖' '😗' '😘' '😙' '😚' '😛' '😜' '😝' '😞' '😟' '😠' '😡' '😢' '😣'
 '😤' '😥' '😦' '😧' '😨' '😩' '😪' '😫' '😬' '😭' '😮' '😯' '😰' '😱' '😲' '😳' '😴' '😵'
 '😶' '😷' '😸' '😹' '😺' '😻' '😼' '😽' '😾' '😿' '🙀' '🙁' '🙂' '🙃' '🙄' '🙅' '🙆' '🙇'
 '🙈' '🙉' '🙊' '🙋' '🙌' '🙍' '🙎' '🙏' '🙐' '🙑' '🙒' '🙓' '🙔' '🙕' '🙖' '🙗' '🙘' '🙙'
 '🙚' '🙛' '🙜' '🙝' '🙞' '🙟' '🙠' '🙡' '🙢' '🙣' '🙤' '🙥' '🙦' '🙧' '🙨' '🙩' '🙪' '🙫'
 '🙬' '🙭' '🙮' '🙯' '🙰' '🙱' '🙲' '🙳' '🙴' '🙵' '🙶' '🙷' '🙸' '🙹' '🙺' '🙻' '🙼' '🙽'
 '🙾' '🙿' '🚀' '🚁' '🚂' '🚃' '🚄' '🚅' '🚆' '🚇' '🚈' '🚉' '🚊' '🚋' '🚌' '🚍' '🚎' '🚏'
 '🚐' '🚑' '🚒' '🚓' '🚔' '🚕' '🚖' '🚗' '🚘' '🚙' '🚚' '🚛' '🚜' '🚝' '🚞' '🚟' '🚠' '🚡'
 '🚢' '🚣' '🚤' '🚥' '🚦' '🚧' '🚨' '🚩' '🚪' '🚫' '🚬' '🚭' '🚮' '🚯' '🚰' '🚱' '🚲' '🚳'
 '🚴' '🚵' '🚶' '🚷' '🚸' '🚹' '🚺' '🚻' '🚼' '🚽' '🚾' '🚿' '🛀' '🛁' '🛂' '🛃' '🛄' '🛅'
 '🛆' '🛇' '🛈' '🛉' '🛊' '🛋' '🛌' '🛍' '🛎' '🛏' '🛐' '🛑' '🛒' '🛓' '🛔' '🛕' '🛖' '🛗'
 '\U0001f6d8' '\U0001f6d9' '\U0001f6da' '\U0001f6db' '\U0001f6dc'
 '\U0001f6dd'

In [15]:
# URL of the raw .npz file
url_train = "https://github.com/psarath22/CS771_M1/raw/main/datasets/train/train_feature.npz"
url_valid = "https://github.com/psarath22/CS771_M1/raw/main/datasets/valid/valid_feature.npz"
url_test = "https://github.com/psarath22/CS771_M1/raw/main/datasets/test/test_feature.npz"
# Download the file
response1 = requests.get(url_train)
response2= requests.get(url_valid)
response3 = requests.get(url_test)

# Save the content to a file
with open('train_feature.npz', 'wb') as f:
    f.write(response1.content)
    print("Downloading TrainData completed!")

# Save the content to a file
with open('valid_feature.npz', 'wb') as f:
    f.write(response2.content)
    print("Downloading validData completed!")

# Save the content to a file
with open('test_feature.npz', 'wb') as f:
    f.write(response3.content)
    print("Downloading TestData completed!")


# read feat dataset
train_feat = np.load("train_feature.npz", allow_pickle=True)
train_feat_X = train_feat['features']
train_feat_Y = train_feat['label']

test_feat_X = np.load("test_feature.npz", allow_pickle=True)['features']

valid_feat = np.load("valid_feature.npz", allow_pickle=True)
valid_feat_X = valid_feat['features']
valid_feat_Y = valid_feat['label']


# Create the training and validation  dataset

train_feat_X = np.array([feat for feat in train_feat_X])
valid_feat_X = np.array([feat for feat in valid_feat_X])
test_feat_X = np.array([feat for feat in test_feat_X])

train_feat_X = np.array([feat.flatten() for feat in train_feat_X])
valid_feat_X = np.array([feat.flatten() for feat in valid_feat_X])
test_feat_X = np.array([feat.flatten() for feat in test_feat_X])

n_components = 150
pca = PCA(n_components=n_components)
train_feat_X = pca.fit_transform(train_feat_X)
valid_feat_X = pca.transform(valid_feat_X)
test_feat_X = pca.transform(test_feat_X)
print(train_feat_X.shape)
print(valid_feat_X.shape)
print(test_feat_X.shape)

(7080, 150)
(489, 150)
(2232, 150)


In [16]:
# read text sequence dataset
train_seq_df = pd.read_csv('https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/train/train_text_seq.csv')
train_seq_X = train_seq_df['input_str'].tolist()
train_seq_Y = train_seq_df['label'].tolist()

test_seq_X = pd.read_csv("https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/test/test_text_seq.csv")['input_str'].tolist()

# read text sequence dataset
valid_seq_df = pd.read_csv("https://raw.githubusercontent.com/psarath22/CS771_M1/main/datasets/valid/valid_text_seq.csv")
valid_seq_X = valid_seq_df['input_str'].tolist()
valid_seq_Y = valid_seq_df['label'].tolist()


# Convert list to NumPy array
train_seq_X = np.array(train_seq_X)
valid_seq_X = np.array(valid_seq_X)
test_seq_X = np.array(test_seq_X)

# Convert labels to numpy arrays
train_seq_Y = np.array(train_seq_Y)
valid_seq_Y = np.array(valid_seq_Y)

# Get unique characters for two digits
unique_characters = [f"{i:02d}" for i in range(100)]  # ['00', '01', ..., '99']
num_classes = len(unique_characters)  # 100 classes

# Create a mapping from characters to indices
char_to_index = {char: idx for idx, char in enumerate(unique_characters)}

# Function to convert input seqs to a 49x100 matrix using overlapping pairs
def one_hot_count_matrix(seq, max_len=50):
    # Initialize a matrix of zeros
    matrix = np.zeros((max_len - 1, num_classes))  # 49x100 matrix for overlapping pairs
    # Extract overlapping pairs and update the matrix
    for i in range(0, max_len - 1):  # Loop from seq[0:2] to seq[48:50]
        char_pair = seq[i:i + 2]  # Get overlapping pairs of characters
        if char_pair in char_to_index:
            index = char_to_index[char_pair]
            matrix[i, index] = 1  # One-hot encoding
    return matrix.flatten()  # Flatten to a vector for SVM input

# Create the training and validation datasets
train_seq_X = np.array([one_hot_count_matrix(seq) for seq in train_seq_X])
valid_seq_X = np.array([one_hot_count_matrix(seq) for seq in valid_seq_X])
test_seq_X = np.array([one_hot_count_matrix(seq) for seq in test_seq_X])

print(train_seq_X.shape)  # Expected shape: (N_train, 4900)
print(valid_seq_X.shape)  # Expected shape: (N_valid, 4900)
print(test_seq_X.shape)  # Expected shape: (N_test, 4900)

(7080, 4900)
(489, 4900)
(2232, 4900)


In [17]:
train_combined_X = np.concatenate((train_emoticon_X, train_feat_X, train_seq_X), axis=1)
valid_combined_X = np.concatenate((valid_emoticon_X, valid_feat_X, valid_seq_X), axis=1)
test_combined_X = np.concatenate((test_emoticon_X, test_feat_X, test_seq_X), axis=1)

print(train_combined_X.shape)
print(valid_combined_X.shape)
print(test_combined_X.shape)

train_combined_Y = train_emoticon_Y
valid_combined_Y = valid_emoticon_Y

num_features = 2938 + 150 + 4900
print(f"no.of features : {num_features}")

n_components = 300
pca = PCA(n_components=n_components)
train_combined_X = pca.fit_transform(train_combined_X)
valid_combined_X = pca.transform(valid_combined_X)
test_combined_X = pca.transform(test_combined_X)
print(train_combined_X.shape)
print(valid_combined_X.shape)
print(test_combined_X.shape)

print(f"reduced no.of features : {n_components}")

(7080, 7988)
(489, 7988)
(2232, 7988)
no.of features : 7988
(7080, 300)
(489, 300)
(2232, 300)
reduced no.of features : 300


In [18]:
#train_combined_XLogistic Regression classifier
logreg_classifier = LogisticRegression(max_iter=1000)
logreg_classifier.fit(train_combined_X, train_combined_Y)

# Predict classes for validation data
valid_predictions_combined = logreg_classifier.predict(valid_combined_X)
train_predictions_combined = logreg_classifier.predict(train_combined_X)
test_predictions_combined = logreg_classifier.predict(test_combined_X)

valid_predictions_combined = valid_predictions_combined.flatten()
train_predictions_combined = train_predictions_combined.flatten()
test_predictions_combined = test_predictions_combined.flatten()

# Calculate training accuracy
train_accuracy_combined = accuracy_score(train_combined_Y, train_predictions_combined)
print(f"Train Accuracy with Logistic Regression for Combined Data: {train_accuracy_combined}")

# Calculate validation accuracy
valid_accuracy_combined = accuracy_score(valid_combined_Y, valid_predictions_combined)
print(f"Validation Accuracy with Logistic Regression for Combined Data: {valid_accuracy_combined}")

f1 = f1_score(valid_combined_Y, valid_predictions_combined, average='weighted')
print(f"F1 Score with Logistic Regression for Combined Data: {f1}")

# Calculate the number of trainable parameters
n_weights = logreg_classifier.coef_.size  # Number of weights (features)
n_intercepts = logreg_classifier.intercept_.size  # Number of intercepts (bias term)

n_trainable_parameters = n_weights + n_intercepts
print(f"Number of trainable parameters for Combined Data: {n_trainable_parameters}")

Train Accuracy with Logistic Regression for Combined Data: 0.9879943502824858
Validation Accuracy with Logistic Regression for Combined Data: 0.9815950920245399
F1 Score with Logistic Regression for Combined Data: 0.9815938592004442
Number of trainable parameters for Combined Data: 301


In [19]:
print(test_predictions_combined)

[1 1 0 ... 0 1 1]


In [20]:
def save_predictions_to_file(predictions, filename):
    with open(filename, 'w') as f:
        for pred in predictions:
            f.write(f"{pred}\n")# saving prediction to text files

save_predictions_to_file(test_predictions_feat, "pred_deepfeat.txt")
save_predictions_to_file(test_predictions_emoticon, "pred_emoticon.txt")
save_predictions_to_file(test_predictions_seq, "pred_textseq.txt")
save_predictions_to_file(test_predictions_combined, "pred_combined.txt")